In [1]:
from multiprocessing import Pool
import numpy as np
import time
from tagger_utils import *
train_data = load_data("data/train_x.csv", "data/train_y.csv")
dev_data = load_data("data/dev_x.csv", "data/dev_y.csv")
test_data = load_data("data/test_x.csv")

100%|██████████| 463/463 [00:00<00:00, 1473.86it/s]


In [140]:

""" Contains the part of speech tagger class. """


def evaluate(data, model):
    """Evaluates the POS model on some sentences and gold tags.

    This model can compute a few different accuracies:
        - whole-sentence accuracy
        - per-token accuracy
        - compare the probabilities computed by different styles of decoding

    You might want to refactor this into several different evaluation functions,
    or you can use it as is. 
    
    As per the write-up, you may find it faster to use multiprocessing (code included). 
    
    """
    processes = 4
    sentences = data[0]
    tags = data[1]
    n = len(sentences)
    k = n//processes
    n_tokens = sum([len(d) for d in sentences])
    unk_n_tokens = sum([1 for s in sentences for w in s if w not in model.word2idx.keys()])
    predictions = {i:None for i in range(n)}
    probabilities = {i:None for i in range(n)}
         
    start = time.time()
    pool = Pool(processes=processes)
    res = []
    for i in range(0, n, k):
        res.append(pool.apply_async(infer_sentences, [model, sentences[i:i+k], i]))
    ans = [r.get(timeout=None) for r in res]
    predictions = dict()
    for a in ans:
        predictions.update(a)
    print(f"Inference Runtime: {(time.time()-start)/60} minutes.")
    
    start = time.time()
    pool = Pool(processes=processes)
    res = []
    for i in range(0, n, k):
        res.append(pool.apply_async(compute_prob, [model, sentences[i:i+k], tags[i:i+k], i]))
    ans = [r.get(timeout=None) for r in res]
    probabilities = dict()
    for a in ans:
        probabilities.update(a)
    print(f"Probability Estimation Runtime: {(time.time()-start)/60} minutes.")


    token_acc = sum([1 for i in range(n) for j in range(len(sentences[i])) if tags[i][j] == predictions[i][j]]) / n_tokens
    unk_token_acc = sum([1 for i in range(n) for j in range(len(sentences[i])) if tags[i][j] == predictions[i][j] and sentences[i][j] not in model.word2idx.keys()]) / unk_n_tokens
    whole_sent_acc = 0
    num_whole_sent = 0
    for k in range(n):
        sent = sentences[k]
        eos_idxes = indices(sent, '.')
        start_idx = 1
        end_idx = eos_idxes[0]
        for i in range(1, len(eos_idxes)):
            whole_sent_acc += 1 if tags[k][start_idx:end_idx] == predictions[k][start_idx:end_idx] else 0
            num_whole_sent += 1
            start_idx = end_idx+1
            end_idx = eos_idxes[i]
    print("Whole sent acc: {}".format(whole_sent_acc/num_whole_sent))
    print("Mean Probabilities: {}".format(sum(probabilities.values())/n))
    print("Token acc: {}".format(token_acc))
    print("Unk token acc: {}".format(unk_token_acc))
    
    confusion_matrix(pos_tagger.tag2idx, pos_tagger.idx2tag, predictions.values(), tags, 'cm.png')

    return whole_sent_acc/num_whole_sent, token_acc, sum(probabilities.values())/n


class POSTagger():
    def __init__(self):
        """Initializes the tagger model parameters and anything else necessary. """
        pass
    
    
    def get_unigrams(self):
        """
        Computes unigrams. 
        Tip. Map each tag to an integer and store the unigrams in a numpy array. 
        """
        n_tags = len(self.tag2idx.keys())
        self.unigram = np.zeros((n_tags))
        for tag in train_data[1]:
            for t in tag:
                tag_idx = self.tag2idx[t]
                self.unigram[tag_idx]= self.unigram[tag_idx]+1
        


    def get_bigrams(self):        
        """
        Computes bigrams. 
        Tip. Map each tag to an integer and store the bigrams in a numpy array
             such that bigrams[index[tag1], index[tag2]] = Prob(tag2|tag1). 
        """
        n_tags = len(self.tag2idx.keys())
        self.bigram = np.zeros((n_tags,n_tags))
        for tag in train_data[1]:
            for i in range(len(tag)-1):
                tag1 = self.tag2idx[tag[i]]
                tag2 = self.tag2idx[tag[i+1]]
                self.bigram[tag1,tag2]= self.bigram[tag1,tag2]+1
        
    
    def get_trigrams(self):
        """
        Computes trigrams. 
        Tip. Similar logic to unigrams and bigrams. Store in numpy array. 
        """
        n_tags = len(self.tag2idx.keys())
        self.trigram = np.zeros((n_tags,n_tags,n_tags))
        tags = train_data[1]
        for tag in train_data[1]:
            for i in range(len(tag)-2):
                tag1 = self.tag2idx[tag[i]]
                tag2 = self.tag2idx[tag[i+1]]
                tag3 = self.tag2idx[tag[i+2]]
                self.trigram[tag1,tag2,tag3]= self.trigram[tag1,tag2,tag3]+1
        
    
    
    def get_emissions(self):
        """
        Computes emission probabilities. 
        Tip. Map each tag to an integer and each word in the vocabulary to an integer. 
             Then create a numpy array such that lexical[index(tag), index(word)] = Prob(word|tag) 
        """
        n_tags = len(self.tag2idx.keys())
        n_words = len(self.word2idx.keys())
        self.lexical = np.zeros((n_tags,n_words))
        words = self.data[0]
        tags = self.data[1]
        n = len(words)
        for x in range(n):
            for i in range(len(words[x])):
                tag_idx = self.tag2idx[tags[x][i]]
                word_idx = self.word2idx[words[x][i]]
                self.lexical[tag_idx,word_idx] = self.lexical[tag_idx,word_idx] + 1
            
    

    def train(self, data):
        """Trains the model by computing transition and emission probabilities.

        You should also experiment:
            - smoothing.
            - N-gram models with varying N.
        
        """
        self.data = data
        self.all_tags = list(set([t for tag in data[1] for t in tag]))
        self.tag2idx = {self.all_tags[i]:i for i in range(len(self.all_tags))}
        self.idx2tag = {v:k for k,v in self.tag2idx.items()}

        self.all_words = list(set([w for word in data[0] for w in word]))
        self.word2idx = {self.all_words[i]:i for i in range(len(self.all_words))}
        self.idx2word = {v:k for k,v in self.word2idx.items()}
    
    def compute_unigram_probability(self, alpha=1):
        self.unigram_prob = (self.unigram + alpha) / (self.unigram.sum() + alpha * len(self.unigram))

    def compute_bigram_probability(self, alpha=1):
        self.bigram_prob = (self.bigram + alpha) / (self.bigram.sum(axis=0, keepdims=True) + alpha * self.bigram.shape[1])

    def compute_trigram_probability(self, alpha=1):
        self.trigram_prob = (self.trigram + alpha) / (self.trigram.sum(axis=(0, 1), keepdims=True) + alpha * self.trigram.shape[2])

    def compute_lexical_probability(self, alpha=1):
        self.lexical_prob = (self.lexical + alpha) / (self.lexical.sum(axis=0, keepdims=True) + alpha * self.lexical.shape[1])

    def get_trigram_probabilty(self, tag1, tag2, tag3):
        tag1_idx = self.tag2idx[tag1]
        tag2_idx = self.tag2idx[tag2]
        tag3_idx = self.tag2idx[tag3]
        return self.trigram_prob[tag1_idx, tag2_idx, tag3_idx]

    def get_bigram_probability(self, tag1, tag2):
        tag1_idx = self.tag2idx[tag1]
        tag2_idx = self.tag2idx[tag2]
        return self.bigram_prob[tag1_idx, tag2_idx]

    def get_unigram_probability(self, tag):
        tag_idx = self.tag2idx[tag]
        return self.unigram_prob[tag_idx]

    def get_lexical_probability(self, tag, word):
        tag_idx = self.tag2idx[tag]
        word_idx = self.word2idx[word]
        return self.lexical_prob[tag_idx, word_idx]

    def word_probability(self, word):
        word_idx = self.word2idx[word]
        return (self.lexical_prob[:, word_idx] * self.unigram_prob).sum()

    def sequence_probability(self, sequence, tags):
        """Computes the probability of a tagged sequence given the emission/transition probabilities."""
        probability = 1
        probability *= self.get_unigram_probability(tags[0]) * self.get_lexical_probability(tags[0], sequence[0]) /  self.word_probability(sequence[0])
        print(probability)
        probability *= self.get_bigram_probability(tags[0], tags[1]) * self.get_lexical_probability(tags[1], sequence[1]) / self.word_probability(sequence[1])
        for i in range(len(sequence) - 2):
            probability *= self.get_trigram_probabilty(tags[i], tags[i+1], tags[i+2]) * self.get_lexical_probability(tags[i+2], sequence[i+2]) / self.word_probability(sequence[i])
        return probability


    def greedy(self, sequence):
        greedy_tags = []
        
        tag1 = max(self.all_tags, 
                key=lambda tag: self.get_unigram_probability(tag) * self.get_lexical_probability(tag, sequence[0]))
        greedy_tags.append(tag1)
        
        tag2 = max(self.all_tags, 
                key=lambda tag: self.get_bigram_probability(greedy_tags[0], tag) * self.get_lexical_probability(tag, sequence[1]))
        greedy_tags.append(tag2)
      
        for i in range(2, len(sequence)):
            tag1 = greedy_tags[i-2]
            tag2 = greedy_tags[i-1]
            
            tag = max(self.all_tags, 
                    key=lambda tag: self.get_trigram_probabilty(tag1, tag2, tag) * self.get_lexical_probability(tag, sequence[i]))
            greedy_tags.append(tag)
        
        return greedy_tags

    def beam_k(self, sequence, k=20):
        k_best = [
            ([tag], self.get_unigram_probability(tag) * self.get_lexical_probability(tag, sequence[0]))
            for tag in sorted(
                self.all_tags,
                key=lambda tag: self.get_unigram_probability(tag) * self.get_lexical_probability(tag, sequence[0]),
                reverse=True
            )[:k]
        ]

        new_k_best = []
        for curr_tags, curr_score in k_best:
            new_k_top_tags = sorted(
                [
                    (curr_tags + [tag], curr_score * self.get_bigram_probability(curr_tags[-1], tag) * self.get_lexical_probability(tag, sequence[1]))
                    for tag in self.all_tags
                ],
                key=lambda x: x[1],
                reverse=True
            )[:k]
            new_k_best.extend(new_k_top_tags)

        k_best = sorted(new_k_best, key=lambda x: x[1], reverse=True)[:k]

        for i in range(2, len(sequence)):
            new_k_best = []
            for curr_tags, curr_score in k_best:
                new_k_top_tags = sorted(
                    [
                        (curr_tags + [tag], curr_score * self.get_trigram_probabilty(curr_tags[-2], curr_tags[-1], tag) * self.get_lexical_probability(tag, sequence[i]))
                        for tag in self.all_tags
                    ],
                    key=lambda x: x[1],
                    reverse=True
                )[:k]
                new_k_best.extend(new_k_top_tags)

            k_best = sorted(new_k_best, key=lambda x: x[1], reverse=True)[:k]
        best_tags = max(k_best, key=lambda x: x[1])[0]
        return best_tags


    def viterbi(self,sequence):
        n_tags = len(self.tag2idx.keys())
        n_words = len(self.word2idx.keys())
        k = len(sequence)
        gamma = np.zeros((k,n_tags)) #most likely tag t after k words (words=observations)
        pre = np.zeros((k,n_tags)) #predecessor tag prev_t that maximized tag t at k
        gamma[0,:] = [self.get_unigram_probability(tag)*self.get_lexical_probability(tag, sequence[0]) for tag in self.all_tags] #prob of tag given word
        tag_idx = np.argmax(gamma[0])
        print(tag_idx, self.idx2tag[tag_idx])
        for x in range(1, k):
            for t2 in self.all_tags:
                t2_idx = self.tag2idx[t2]
                gamma_probs = [
                    (self.tag2idx[t1], gamma[x-1, self.tag2idx[t1]] * self.get_bigram_probability(t1, t2) * self.get_lexical_probability(t2, sequence[x]))
                    for t1 in self.all_tags
                ]
                max_gamma_prob = max(gamma_probs, key=lambda item: item[1])
                gamma[x, t2_idx] = max_gamma_prob[1]
                pre[x, t2_idx] = max_gamma_prob[0]
        
        
        tag_idx = np.argmax(gamma[k-1])
        viterbi_tags = [self.idx2tag[tag_idx]]
        for x in range(1,k):
            tag_idx = int(pre[k-x, tag_idx])
            viterbi_tags = [self.idx2tag[tag_idx]] + viterbi_tags

        return viterbi_tags 
      
    def inference(self, sequence):
        """Tags a sequence with part of speech tags.

        You should implement different kinds of inference (suggested as separate
        methods):

            - greedy decoding
            - decoding with beam search
            - viterbi
        """
        ## TODO
        greedy_tags = self.greedy(sequence)
        print(greedy_tags)
        beam_tags = self.beam_k(sequence)
        print(beam_tags)

        viterbi_tags = self.viterbi(sequence)
        print(viterbi_tags)


            
        return []


if __name__ == "__main__":
    pos_tagger = POSTagger()

    train_data = train_data
    dev_data = dev_data
    test_data = test_data

    pos_tagger.train(train_data)
    pos_tagger.get_unigrams()
    pos_tagger.get_bigrams()
    pos_tagger.get_trigrams()
    pos_tagger.get_emissions()
    pos_tagger.compute_unigram_probability()
    pos_tagger.compute_bigram_probability()
    pos_tagger.compute_trigram_probability()
    pos_tagger.compute_lexical_probability()
    seq = ['I', 'am','old',"but","I","love", "you"]
    prob = pos_tagger.word_probability("I")
    print(prob)
    # prob = pos_tagger.sequence_probability(seq,['PRP', 'VBP', 'JJ'])
    
    print(prob)

    pos_tagger.inference(seq)
    # Experiment with your decoder using greedy decoding, beam search, viterbi...


    # Here you can also implement experiments that compare different styles of decoding,
    # smoothing, n-grams, etc.
    # evaluate(dev_data, pos_tagger)

    # # Predict tags for the test set
    # test_predictions = []
    # for sentence in test_data:
    #     test_predictions.extend(pos_tagger.inference(sentence))
    
    # Write them to a file to update the leaderboard
    # TODO


0.0004614798929380484
0.0004614798929380484
['PRP', 'VBP', 'JJ', 'CC', 'PRP', 'VBP', 'PRP']
['PRP', 'VBP', 'JJ', 'CC', 'PRP', 'VBP', 'PRP']
6 PRP
['PRP', 'VBP', 'JJ', 'CC', 'PRP', 'VBP', 'PRP']


In [75]:
print(train_data[0][0][:7])
print(train_data[1][0][:7])

['-docstart-', 'Pierre', 'Vinken', ',', '61', 'years', 'old']
['O', 'NNP', 'NNP', ',', 'CD', 'NNS', 'JJ']


In [65]:
y = np.zeros((100,2,1))
y.sum(axis=0)


array([[0.],
       [0.]])

In [73]:
pos_tagger.bigram.sum(axis=0)

array([1.9120e+03, 9.1000e+01, 2.3940e+03, 5.1250e+03, 1.6170e+04,
       1.4070e+03, 1.2837e+04, 6.4000e+01, 3.2170e+03, 7.2097e+04,
       4.3879e+04, 3.5475e+04, 1.2570e+03, 2.0490e+03, 0.0000e+00,
       2.8837e+04, 1.0300e+03, 7.2040e+03, 9.6805e+04, 6.7431e+04,
       4.5356e+04, 6.0650e+03, 1.4622e+04, 1.5660e+03, 3.0000e+01,
       6.4500e+02, 2.6323e+04, 3.2000e+02, 1.0210e+03, 5.9928e+04,
       1.9353e+04, 1.2900e+02, 5.2290e+03, 2.9600e+02, 1.7410e+03,
       6.4780e+03, 1.1013e+04, 2.2445e+04, 1.6250e+04, 5.1050e+03,
       3.6020e+03, 2.0400e+02, 2.1357e+04, 3.7000e+01, 1.7297e+04,
       1.3870e+03, 9.3950e+03])

In [48]:
    
    def beam_k(self,sequence):
        k = 5
        k_best = [(tag, self.unigram[self.tag2idx[tag]] * self.lexical[self.tag2idx[tag], self.word2idx[sequence[0]]]) 
            for tag in sorted(self.all_tags, key=lambda tag: self.unigram[self.tag2idx[tag]] * self.lexical[self.tag2idx[tag], self.word2idx[sequence[0]]], reverse=True)[:k]]

        new_k_best = []
        for curr_tags, curr_score in k_best:
            new_k_top_tags = sorted(
                [(curr_tags, tag, curr_score * self.bigram[self.tag2idx[curr_tags[0]], self.tag2idx[tag]] * self.lexical[self.tag2idx[tag], self.word2idx[sequence[1]]]) 
                for tag in self.all_tags],
                key=lambda x: x[2],  
                reverse=True
            )[:k] 
            new_k_best.extend(new_k_top_tags)

        new_k_best = sorted(new_k_best, key=lambda x: x[2], reverse=True)[:k]
        k_best = new_k_best

        for i in range(2,len(sequence)):
            new_k_best = []
            for curr_tags, curr_score in k_best:
                new_k_top_tags = sorted(
                    [(curr_tags, tag, curr_score * self.trigram[self.tag2idx[curr_tags[-2]],self.tag2idx[curr_tags[-1]], self.tag2idx[tag]] * self.lexical[self.tag2idx[tag], self.word2idx[sequence[i]]]) 
                    for tag in self.all_tags],
                    key=lambda x: x[2],  
                    reverse=True
                )[:k] 
                new_k_best.extend(new_k_top_tags)
            k_best = new_k_best
       
        
     

array([nan, nan, nan, nan,  1., nan, nan, nan, nan,  1.,  1.,  1., nan,
       nan, nan,  1., nan,  1.,  1.,  1.,  1., nan, nan, nan, nan, nan,
        1., nan, nan,  1.,  1., nan, nan, nan, nan, nan,  1., nan, nan,
       nan, nan, nan, nan, nan, nan,  1., nan])